# Pair Programming ETL Carga I

Igual que en el ejercicio de pair programming de ETL Transformación II, tendréis que crear una clase que nos permita cargar los datos en SQL I.
Happy coding 🦈

In [130]:
import pandas as pd
import mysql.connector

In [131]:
df_ataques = pd.read_csv("datos/4.ataques_limpio.csv", index_col=0)
df_ataques.head(1)

,case_number,year,type,country,date,mes,fatal,sexo,especie_cat,age
0,2018.06.25,2018,Boating,usa,25-Jun-2018,Jun,n,F,White,57.0


In [132]:
df_clima= pd.read_csv("files/1.paises_meteo_info.csv", index_col=0)
df_clima.head(1)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 15}, {'layer': '900m...","[{'layer': '950mb', 'direction': 85, 'speed': ...",25,-1,12,1010,none,1,0,105,3,Papua New Guinea


In [133]:
class Cargar:
    
    def __init__(self, nombre_bbdd, contraseña):
        # nuestra clase va a recibir dos parámetros que son fijos a lo largo de toda la BBDD, el nombre de la BBDD y la contraseña con el servidor. 
        self.nombre_bbdd = nombre_bbdd
        self.contraseña = contraseña

    # método para crear la BBDD 
    def crear_bbdd(self):

        mydb = mysql.connector.connect(host="localhost",
                                       user="root",
                                       password=f'{self.contraseña}') 
        mycursor = mydb.cursor()
        print("Conexión realizada con éxito")
        print("La base de datos se ha creado correctamente")

        try:
            mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.nombre_bbdd};")
            
        except:
            print("La BBDD ya existe")
    
    # método para crear tablas  e insertar datos en ellas   
    def crear_insertar_tabla(self, query):
        
        mydb = mysql.connector.connect(host="localhost",
                                       user="root",
                                       password=f'{self.contraseña}', 
                                       database=f"{self.nombre_bbdd}") 
        mycursor = mydb.cursor()
        
        try:
            mycursor.execute(query)
            mydb.commit()
          
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)
  

Llamamos a la clase

In [134]:
db_sql = Cargar("tiburones2", "AlumnaAdalab")

Aplicamos el método crear_bbdd para crear la base de datos

In [135]:
mydb = db_sql.crear_bbdd()

Conexión realizada con éxito
La base de datos se ha creado correctamente


Aplicamos el método para crear las tablas. Con el método crear_tabla(query)

In [136]:
# query crear tabla ataques
tabla_ataques = '''
CREATE TABLE IF NOT EXISTS `tiburones2`.`ataques` (
  `id` INT NOT NULL AUTO_INCREMENT,
  `case_number` VARCHAR(2000) DEFAULT NULL, 
  `year` INT DEFAULT NULL , 
  `type` VARCHAR(2000) DEFAULT NULL, 
  `country` VARCHAR(2000) DEFAULT NULL, 
  `date` VARCHAR(2000) DEFAULT NULL,
  `mes` VARCHAR(2000) DEFAULT NULL,
  `fatal` VARCHAR(2000) DEFAULT NULL,
  `sexo` VARCHAR(2000) DEFAULT NULL,
  `especie_cat`VARCHAR(50) DEFAULT NULL,
  `age` FLOAT (6),
  
  PRIMARY KEY (`id`))
ENGINE = InnoDB;
'''
# El ENGINE, es el motor de almacenamiento, el encargado de almacenar y manejar la información de la tabla.

In [137]:
# query crear tabla clima

tabla_clima = '''
CREATE TABLE IF NOT EXISTS `tiburones2`.`clima`(
  `id` INT NOT NULL AUTO_INCREMENT,
  `timepoint` INT NOT NULL, 
  `cloudcover` INT NOT NULL, 
  `highcloud` INT NOT NULL,
  `midcloud` INT NOT NULL, 
  `lowcloud` INT NOT NULL, 
  `temp2m` INT NOT NULL, 
  `lifted_index` INT NOT NULL, 
  `rh2m` INT NOT NULL,
  `msl_pressure` INT NOT NULL, 
  `prec_amount` INT (50) NOT NULL, 
  `snow_depth` INT (50) NOT NULL, 
  `wind10m.direction` INT NOT NULL,
  `wind10m.speed` INT NOT NULL,
  `pais` VARCHAR (50) NOT NULL,
    
  PRIMARY KEY ( `id`),

  CONSTRAINT `fk_clima_ataques`
    FOREIGN KEY (`id`)
    REFERENCES `ataques` (`id`))
ENGINE = InnoDB;
'''

In [138]:
db_sql.crear_insertar_tabla(tabla_ataques)

In [139]:
db_sql.crear_insertar_tabla(tabla_clima)

In [140]:
for indice, fila in df_ataques.iterrows():
    try:
        query_ataques = f""" INSERT INTO ataques (`case_number`, `year`, `type`, `country`, `date`, `mes`, `fatal`, `sexo`, `especie_cat`, `age`)
                                VALUES ("{fila['case_number']}", "{fila['year']}", "{fila['type']}", "{fila['country']}", "{fila['date']}", "{fila['mes']}","{fila['fatal']}", "{fila['sexo']}","{fila['especie_cat']}", "{fila['age']}");
                                """
        # Llamamos a la función dentro del for, ya que si no, no itera por todos los valores y sólo nos inserta la primera línea.
        db_sql.crear_insertar_tabla(query_ataques)
    
    except:
        pass

1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'During the war" 1943-1945", "During","n", "M","nan", "28.0")' at line 2
Error Code: 1064
SQLSTATE 42000
Message You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'During the war" 1943-1945", "During","n", "M","nan", "28.0")' at line 2
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'Before the war"", "Before","n", "M","nan", "28.0")' at line 2
Error Code: 1064
SQLSTATE 42000
Message You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'Before the war"", "Before","n", "M","nan", "28.0")' at line 2


In [141]:
for indice, fila in df_clima.iterrows():
    query_clima = f""" INSERT INTO `clima`( `timepoint`, `cloudcover`, `highcloud`, `midcloud`, `lowcloud`, `temp2m`, `lifted_index`, `rh2m`, `msl_pressure`, `prec_amount`, `snow_depth`, `wind10m.direction`, `wind10m.speed`, `pais`)
                        VALUES ('{fila["timepoint"]}', '{fila["cloudcover"]}', '{fila["highcloud"]}', '{fila["midcloud"]}', '{fila["lowcloud"]}', '{fila["temp2m"]}', '{fila["lifted_index"]}', '{fila["rh2m"]}', '{fila["msl_pressure"]}', '{fila["prec_amount"]}', '{fila["snow_depth"]}', '{fila["wind10m.direction"]}', '{fila["wind10m.speed"]}', '{fila["pais"]}');
                    """
    # Llamamos a la función dentro del for, ya que si no, no itera por todos los valores y sólo nos inserta la primera línea.
    db_sql.crear_insertar_tabla(query_clima)